In [ ]:
!pip install cohere
!pip install -q cassio datasets langchain openai tiktoken

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.7/51.7 kB 810.6 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 25.4 MB/s eta 0:00:00
  Attempting uninstall: importlib_metadata
    Found existing installation: importlib-metadata 7.0.0
    Uninstalling importlib-metadata-7.0.0:
      Successfully uninstalled importlib-metadata-7.0.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llmx 0.0.15a0 requires openai, which is not installed.
llmx 0.0.15a0 requires tiktoken, which is not installed.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.1/40.1 kB 813.1 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 10.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 809.1/809.1 kB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 223.7/223.7 kB 11.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━

In [ ]:
# LangChain components to use
from langchain.vectorstores.cassandra import Cassandra
from langchain.indexes.vectorstore import VectorStoreIndexWrapper
from langchain.llms import OpenAI
from langchain.embeddings import OpenAIEmbeddings

# Support for dataset retrieval with Hugging Face
from datasets import load_dataset

# With CassIO, the engine powering the Astra DB integration in LangChain,
# you will also initialize the DB connection:
import cassio

In [ ]:
!pip install PyPDF2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 4.3 MB/s eta 0:00:00


In [ ]:
from PyPDF2 import PdfReader

In [ ]:
ASTRA_DB_APPLICATION_TOKEN = "" # enter the "AstraCS:..." string found in in your Token JSON file
ASTRA_DB_ID = "" # enter your Database ID

OPENAI_API_KEY = "" # enter your OpenAI key

In [ ]:
# provide the path of  pdf file/files.
pdfreader = PdfReader('chatgpt.pdf')

In [ ]:
from typing_extensions import Concatenate
# read text from pdf
raw_text = ''
for i, page in enumerate(pdfreader.pages):
    content = page.extract_text()
    if content:
        raw_text += content

In [ ]:
raw_text

"Introduction  \nChatGPT , developed by OpenAI, represents a groundbreaking advancement in the realm of artificial \nintelligence. This sophisticated language model, rooted in the GPT (Generative Pretrained \nTransformer) architecture, has revolutionized the way we interact with AI. With its capacity to \nunderstand and generate human -like text, ChatGPT is not just a technological marvel but a bridge \nbetween human cognition and artificial intelligence, offering a glimpse into the future of human -AI \ninteraction.   \n \nDevelopment and Training  \nThe development of ChatGPT involved extensive training using vast datasets composed of diverse \ntext sources. This process, known as machine learning, enables the model to predict and generate \ntext based on the patterns it has learn ed. The underlying Transformer architecture is particularly \nadept at processing natural language, allowing ChatGPT to understand context, nuance, and even \ncomplex language structures. This rigorous trai

In [ ]:
cassio.init(token=ASTRA_DB_APPLICATION_TOKEN, database_id=ASTRA_DB_ID)

ERROR:cassandra.connection:Closing connection <AsyncoreConnection(135418530944416) 74c00cf7-8583-47a2-bb01-d52916370920-us-east1.db.astra.datastax.com:29042:f7b8d1d5-5e9f-4448-a072-82016e7b8d90> due to protocol error: Error from server: code=000a [Protocol error] message="Beta version of the protocol used (5/v5-beta), but USE_BETA flag is unset"


In [ ]:
llm = OpenAI(openai_api_key=OPENAI_API_KEY)
embedding = OpenAIEmbeddings(openai_api_key=OPENAI_API_KEY)

In [ ]:
astra_vector_store = Cassandra(
    embedding=embedding,
    table_name="qa_mini_demo",
    session=None,
    keyspace=None,
)

In [ ]:
from langchain.text_splitter import CharacterTextSplitter
# We need to split the text using Character Text Split such that it sshould not increse token size
text_splitter = CharacterTextSplitter(
    separator = "\n",
    chunk_size = 800,
    chunk_overlap  = 200,
    length_function = len,
)
texts = text_splitter.split_text(raw_text)

In [ ]:
texts[:50]

['Introduction  \nChatGPT , developed by OpenAI, represents a groundbreaking advancement in the realm of artificial \nintelligence. This sophisticated language model, rooted in the GPT (Generative Pretrained \nTransformer) architecture, has revolutionized the way we interact with AI. With its capacity to \nunderstand and generate human -like text, ChatGPT is not just a technological marvel but a bridge \nbetween human cognition and artificial intelligence, offering a glimpse into the future of human -AI \ninteraction.   \n \nDevelopment and Training  \nThe development of ChatGPT involved extensive training using vast datasets composed of diverse \ntext sources. This process, known as machine learning, enables the model to predict and generate',
 "The development of ChatGPT involved extensive training using vast datasets composed of diverse \ntext sources. This process, known as machine learning, enables the model to predict and generate \ntext based on the patterns it has learn ed. The

In [ ]:
astra_vector_store.add_texts(texts[:50])

print("Inserted %i headlines." % len(texts[:50]))

astra_vector_index = VectorStoreIndexWrapper(vectorstore=astra_vector_store)

Inserted 5 headlines.


In [ ]:
first_question = True
while True:
    if first_question:
        query_text = input("\nEnter your question (or type 'quit' to exit): ").strip()
    else:
        query_text = input("\nWhat's your next question (or type 'quit' to exit): ").strip()

    if query_text.lower() == "quit":
        break

    if query_text == "":
        continue

    first_question = False

    print("\nQUESTION: \"%s\"" % query_text)
    answer = astra_vector_index.query(query_text, llm=llm).strip()
    print("ANSWER: \"%s\"\n" % answer)

    print("FIRST DOCUMENTS BY RELEVANCE:")
    for doc, score in astra_vector_store.similarity_search_with_score(query_text, k=4):
        print("    [%0.4f] \"%s ...\"" % (score, doc.page_content[:84]))


QUESTION: "applications of chatgpt"


ANSWER: "ChatGPT has found its place in content creation, from writing articles to composing poetry, as well as customer service, education, and bridging language barriers."

FIRST DOCUMENTS BY RELEVANCE:


    [0.9287] "Capabilities and Applications  
ChatGPT's capabilities extend far beyond basic text  ..."
    [0.9261] "ChatGPT’s global influence is noteworthy. It has the potential to bridge language ba ..."
    [0.9256] "In conclusion, ChatGPT stands as a remarka ble achievement in the field of artificia ..."
    [0.9236] "The development of ChatGPT involved extensive training using vast datasets composed  ..."

QUESTION: "limitations of chatgpt"


ANSWER: "One significant limitation of ChatGPT is its knowledge cutoff in April 2023, meaning it cannot access or provide information beyond that point. Additionally, as a machine learning model, it sometimes might generate incorrect or biased information."

FIRST DOCUMENTS BY RELEVANCE:


    [0.9311] "Limitations and Ethical Considerations  
Despite its advanced capabilities, ChatGPT  ..."
    [0.9289] "Capabilities and Applications  
ChatGPT's capabilities extend far beyond basic text  ..."
    [0.9248] "In conclusion, ChatGPT stands as a remarka ble achievement in the field of artificia ..."
    [0.9207] "ChatGPT’s global influence is noteworthy. It has the potential to bridge language ba ..."
